In [6]:
#import dependencies
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
#from KaggleWord2VecUtility import KaggleWord2VecUtility
from gensim.models import Word2Vec
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import numpy as np


In [7]:
#Reading the CSV file

df = pd.read_csv('../Data/bitcointweets.csv',header=None)
#df.rename(columns={0: ‘DateTime’, 1: ‘Tweet’, 2: 'Handle', 3:'Followers', 4:'Unknown', 5:'Hashtags', 6:'Ref', 7:'Sentiment'}, inplace=True)
df.columns =["DateTime","Tweet","Handle","Followers","Unknown","Hashtags","Ref", "Sentiment"]
df.head()

,DateTime,Tweet,Handle,Followers,Unknown,Hashtags,Ref,Sentiment
0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,16522,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral']
1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,1295,0,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral']
2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,6090,0,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive']
3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,2626,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive']
4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,184,0,[],"<a href=""http://twitter.com/download/android"" ...",['positive']


In [8]:
df['Sentiment'] = df['Sentiment'].replace(
    {"['positive']": 'positive', "['negative']": 'negative',"['neutral']": 'neutral'})
df.head()

,DateTime,Tweet,Handle,Followers,Unknown,Hashtags,Ref,Sentiment
0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,16522,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",neutral
1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,1295,0,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",neutral
2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,6090,0,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",positive
3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,2626,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",positive
4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,184,0,[],"<a href=""http://twitter.com/download/android"" ...",positive


In [9]:
from sklearn import preprocessing
encoder= preprocessing.LabelEncoder()
categories=["positive","negative","neutral"]
encoder.fit(categories)
#new_categories=["negative","positive", "positive","neutral"]
# encoder.transform(new_categories)

LabelEncoder()

In [10]:
df['Sentiment']=encoder.transform(df['Sentiment'])
df.head()
#df['Sentiment'].unique()

,DateTime,Tweet,Handle,Followers,Unknown,Hashtags,Ref,Sentiment
0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,16522,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1
1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,1295,0,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",1
2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,6090,0,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2
3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,2626,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2
4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,184,0,[],"<a href=""http://twitter.com/download/android"" ...",2


In [11]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split
X= df['Tweet']
y= df['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
y_train


31289    2
22712    2
30538    1
13802    2
2130     2
15975    0
42871    1
44149    1
7887     0
9927     1
21725    2
29201    0
10465    0
13146    1
6878     2
31088    1
13448    2
33719    1
24053    2
40925    1
27631    2
42958    1
45521    2
49721    2
10492    2
30222    1
11859    2
45084    0
49875    0
14473    1
        ..
26103    2
17137    1
19433    1
24535    1
31959    2
27204    2
49714    1
22926    1
37532    1
19946    1
8444     1
35730    2
45413    1
21758    2
32025    1
36230    2
31228    0
43757    2
7751     2
49100    1
20609    1
32912    2
21440    2
7813     2
43723    1
50057    1
32511    1
5192     2
12172    2
33003    2
Name: Sentiment, Length: 38144, dtype: int64

In [12]:
# This function converts a text to a sequence of words
def review_wordlist(review, remove_stopwords=False):
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    # 2. Removing non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # 3. Converting to lower case and splitting
    words = review_text.lower().split()
    # 4. Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
    
    return(words)

In [13]:
# word2vec expects list of lists
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/robertsayler/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [14]:
# word2vec expects a list of lists.
# Using punkt tokenizer for better splitting of a paragraph into sentences.

import nltk.data
nltk.download('popular')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/robertsayler/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/robertsayler/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/robertsayler/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/robertsayler/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/robertsayler/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/robertsayler/nltk_data...
[nltk_data]    

In [15]:
# This function splits a review into sentences
def review_sentences(review, tokenizer, remove_stopwords=False):
    # 1. Using nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence,\
                                            remove_stopwords))

    # This returns the list of lists
    return sentences

In [16]:
sentences = []
print("Parsing sentences from training set")
for review in X_train:
    sentences += review_sentences(review, tokenizer)

Parsing sentences from training set


/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/IdiDYmQPWr" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/BzOn01gBE0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % d

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/d94XMySJ0Q" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/SGlzX48XXv" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/iFuRZ1kkoQ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful S

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/ClBT…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/lGOFqiMquX" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/nwCMdVYMB0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.'

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/nAirjRm6gs" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/KQGyRgfzJ6" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/hIjvkS6rPC…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful 

...

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/a8JARKCKjp" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/7FFpE21muM" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/pqQMiJdZUT" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful S

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/PShnCERspv" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/7j6wYQzGAd" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/KUW1npHdHd" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful S

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/buUywiBHEx" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/ek3ZJjqoCQ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/ZNCIm0KhRs" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful S

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/pnUY7v9yPr" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/lZGNHXj…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/3zwTGDfF9m" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Sou

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/BHeXmW8Fvk" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/6UDHILY0Px" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:335: UserWarning: "https://t.co/wVaJJJalhN" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful S

In [17]:
# Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [18]:
# Creating the model and setting values for the various parameters
num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "Final_300features_40minwords_10context.h5"
model.save(model_name)

2019-07-06 13:15:50,269 : INFO : collecting all words and their counts
2019-07-06 13:15:50,270 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-06 13:15:50,308 : INFO : PROGRESS: at sentence #10000, processed 117101 words, keeping 12789 word types
2019-07-06 13:15:50,366 : INFO : PROGRESS: at sentence #20000, processed 234291 words, keeping 20098 word types
2019-07-06 13:15:50,443 : INFO : PROGRESS: at sentence #30000, processed 351820 words, keeping 26475 word types


Training model....


2019-07-06 13:15:50,518 : INFO : PROGRESS: at sentence #40000, processed 469518 words, keeping 32165 word types
2019-07-06 13:15:50,568 : INFO : PROGRESS: at sentence #50000, processed 587785 words, keeping 37619 word types
2019-07-06 13:15:50,620 : INFO : PROGRESS: at sentence #60000, processed 704598 words, keeping 42693 word types
2019-07-06 13:15:50,647 : INFO : collected 44912 word types from a corpus of 754151 raw words and 64277 sentences
2019-07-06 13:15:50,647 : INFO : Loading a fresh vocabulary
2019-07-06 13:15:50,697 : INFO : min_count=40 retains 1608 unique words (3% of original 44912, drops 43304)
2019-07-06 13:15:50,699 : INFO : min_count=40 leaves 636874 word corpus (84% of original 754151, drops 117277)
2019-07-06 13:15:50,707 : INFO : deleting the raw counts dictionary of 44912 items
2019-07-06 13:15:50,712 : INFO : sample=0.001 downsamples 48 most-common words
2019-07-06 13:15:50,718 : INFO : downsampling leaves estimated 433869 word corpus (68.1% of prior 636874)
201

In [19]:
# This will give the total number of words in the vocabolary created from this dataset
model.wv.syn0.shape

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


(1608, 300)

In [15]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [16]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [17]:
# Calculating average feature vector for training set
clean_train_reviews = []
for review in X_train:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

Review 0 of 38144
Review 1000 of 38144


C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 2000 of 38144
Review 3000 of 38144
Review 4000 of 38144
Review 5000 of 38144
Review 6000 of 38144
Review 7000 of 38144
Review 8000 of 38144
Review 9000 of 38144
Review 10000 of 38144
Review 11000 of 38144
Review 12000 of 38144
Review 13000 of 38144
Review 14000 of 38144
Review 15000 of 38144
Review 16000 of 38144
Review 17000 of 38144
Review 18000 of 38144
Review 19000 of 38144
Review 20000 of 38144
Review 21000 of 38144
Review 22000 of 38144
Review 23000 of 38144
Review 24000 of 38144
Review 25000 of 38144
Review 26000 of 38144
Review 27000 of 38144
Review 28000 of 38144
Review 29000 of 38144
Review 30000 of 38144
Review 31000 of 38144
Review 32000 of 38144
Review 33000 of 38144
Review 34000 of 38144
Review 35000 of 38144
Review 36000 of 38144
Review 37000 of 38144
Review 38000 of 38144


In [18]:
# Calculating average feature vactors for test set     
clean_test_reviews = []
for review in X_test:
    clean_test_reviews.append(review_wordlist(review,remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 12715
Review 1000 of 12715
Review 2000 of 12715


C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 3000 of 12715
Review 4000 of 12715
Review 5000 of 12715
Review 6000 of 12715
Review 7000 of 12715
Review 8000 of 12715
Review 9000 of 12715
Review 10000 of 12715
Review 11000 of 12715
Review 12000 of 12715


In [19]:
trainDataVecs.shape

(38144, 300)

In [20]:
# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
    
print("Fitting random forest to training data....")    
forest = forest.fit(trainDataVecs, y_train)
forest

Fitting random forest to training data....


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [33]:
#forest.score(X_train, y_train)

In [34]:
forest.score(trainDataVecs, y_train)

0.9969064597315436

In [35]:
forest.score(testDataVecs, y_test)

0.8990955564294141

In [23]:
# Predicting the sentiment values for test data and saving the results in a csv file 
result = forest.predict(testDataVecs)
result

array([2, 1, 1, ..., 1, 2, 1])

In [39]:
output = pd.DataFrame(data={"Tweet":X_test,"sentiment":result})
output.head()

,Tweet,sentiment
44969,🎉 20$ for free registering to Binance 🎉\n\n0.1...,2
10916,This might be noteworthy. https://t.co/lQWMLT8EKX,1
11708,Bitcoin | Ripple | Litecoin | Neo | Cardano\n\...,1
23639,"RT @CoinbayExchange: Dear #bountyhunters 💸 , H...",1
14274,The Image MASSter Solo 101 Forensic Hard Drive...,1


In [42]:
#output.to_csv( "output.csv", index=False, quoting=3 )
output.to_csv("Output/output.csv", index=False, header=True)